# Emojify

Let's try to build a model that, given an short text sentence as input, tries to find an adequate emoji.


In [3]:
# base packages
import numpy as np
from utils import *
import emoji
import matplotlib.pyplot as plt

%matplotlib inline

## Emojifier-V1

This first version of the Emojifer is going to be our baseline model, the model performs the following steps:
- takes as input a sentence
- finds the vector representation for each word in the sentence, and average these vetors into one
- feeds this average vector to a dense layer with a softmax activation for classification

### Data Preprocessing

In [2]:
X_train, Y_train = read_csv('data/train_emoji.csv')
X_test, Y_test = read_csv('data/tesss.csv')

We need to find the largest sentence for 0 padding all others to have the same size.

In [3]:
maxLen = len(max(X_train, key=lambda x: len(x.split())).split())

Let's look at a sample from our dataset.

In [4]:
for idx in range(10):
    print(X_train[idx], label_to_emoji(Y_train[idx]))

never talk to me again 😞
I am proud of your achievements 😄
It is the worst day in my life 😞
Miss you so much ❤️
food is life 🍴
I love you mum ❤️
Stop saying bullshit 😞
congratulations on your acceptance 😄
The assignment is too long  😞
I want to go play ⚾


We need to one hot encode the target vector to be appropriate with the 'softmax' activation.

In [5]:
Y_oh_train = convert_to_one_hot(Y_train, C = 5)
Y_oh_test = convert_to_one_hot(Y_test, C = 5)

In [6]:
idx = 50
print(f"Sentence '{X_train[idx]}' has label index {Y_train[idx]}, which is emoji {label_to_emoji(Y_train[idx])}", )
print(f"Label index {Y_train[idx]} in one-hot encoding format is {Y_oh_train[idx]}")

Sentence 'I missed you' has label index 0, which is emoji ❤️
Label index 0 in one-hot encoding format is [1. 0. 0. 0. 0.]


### Build Model

In [7]:
word_to_index, index_to_word, word_to_vec_map = read_glove_vecs('data/glove.6B.50d.txt')

Let's look at our dictionaries.

In [8]:
word = "cucumber"
idx = 289846
print("the index of", word, "in the vocabulary is", word_to_index[word])
print("the", str(idx) + "th word in the vocabulary is", index_to_word[idx])

the index of cucumber in the vocabulary is 113317
the 289846th word in the vocabulary is potatos


We need a function that converts the input sentence to the average vector that is going to feed into the softmax dense layer.

In [9]:
def sentence_to_avg(sentence, word_to_vec_map):
    
    any_word = list(word_to_vec_map.keys())[0]
    
    words = sentence.lower().split()

    avg = np.zeros(word_to_vec_map[any_word].shape)
    
    count = 0
    
    for w in words:
        if w in word_to_vec_map:
            avg += word_to_vec_map[w]
            count +=1
          
    if count > 0:
        avg = avg / count
    
    return avg

In [10]:
avg = sentence_to_avg("Morrocan couscous is my favorite dish", word_to_vec_map)
print("avg = \n", avg)

avg = 
 [-0.008005    0.56370833 -0.50427333  0.258865    0.55131103  0.03104983
 -0.21013718  0.16893933 -0.09590267  0.141784   -0.15708967  0.18525867
  0.6495785   0.38371117  0.21102167  0.11301667  0.02613967  0.26037767
  0.05820667 -0.01578167 -0.12078833 -0.02471267  0.4128455   0.5152061
  0.38756167 -0.898661   -0.535145    0.33501167  0.68806933 -0.2156265
  1.797155    0.10476933 -0.36775333  0.750785    0.10282583  0.348925
 -0.27262833  0.66768    -0.10706167 -0.283635    0.59580117  0.28747333
 -0.3366635   0.23393817  0.34349183  0.178405    0.1166155  -0.076433
  0.1445417   0.09808667]


Now we can build the model from scratch.

In [11]:
def model(X, Y, word_to_vec_map, learning_rate = 0.01, num_iterations = 400):

    any_word = list(word_to_vec_map.keys())[0]
    
    m = Y.shape[0]                             # number of training examples
    n_y = len(np.unique(Y))                    # number of classes  
    n_h = word_to_vec_map[any_word].shape[0]   # dimensions of the GloVe vectors 
    
    W = np.random.randn(n_y, n_h) / np.sqrt(n_h)
    b = np.zeros((n_y,))
    
    Y_oh = convert_to_one_hot(Y, C = n_y) 
    
    for t in range(num_iterations):
        
        cost = 0
        dW = 0
        db = 0
        
        for i in range(m):
            
            avg = sentence_to_avg(X[i], word_to_vec_map)

            z = np.dot(W, avg) + b
            a = softmax(z)

            cost += - np.sum(Y_oh[i] * a)
    
            dz = a - Y_oh[i]
            dW += np.dot(dz.reshape(n_y,1), avg.reshape(1, n_h))
            db += dz

            W = W - learning_rate * dW
            b = b - learning_rate * db
            
        if t % 100 == 0:
            print("Epoch: " + str(t) + " --- cost = " + str(cost))
            pred = predict(X, Y, W, b, word_to_vec_map)

    return pred, W, b

Let's train our baseline model.

In [12]:
np.random.seed(1)
pred, W, b = model(X_train, Y_train, word_to_vec_map)

Epoch: 0 --- cost = -47.92582129545767
Accuracy: 0.5454545454545454
Epoch: 100 --- cost = -124.63320353765374
Accuracy: 0.9318181818181818
Epoch: 200 --- cost = -131.30328140693368
Accuracy: 1.0
Epoch: 300 --- cost = -131.69623486452446
Accuracy: 1.0


### Evaluate Model

In [14]:
print("Training set:")
pred_train = predict(X_train, Y_train, W, b, word_to_vec_map)
print('Test set:')
pred_test = predict(X_test, Y_test, W, b, word_to_vec_map)

Training set:
Accuracy: 1.0
Test set:
Accuracy: 0.9107142857142857


The model does a very good job compared to random guessing that would have an accuracy of only 20%.  
Note how the performance is good with only a few training examples, this is because we are using pre-trained word embeddings where words carry significant meaning.

In [15]:
X_my_sentences = np.array(["i treasure you", "i love you", "funny lol", "lets play with a ball", "food is ready", "today is not good"])
Y_my_labels = np.array([[0], [0], [2], [1], [4],[3]])

pred = predict(X_my_sentences, Y_my_labels , W, b, word_to_vec_map)
print_predictions(X_my_sentences, pred)

Accuracy: 0.8333333333333334

i treasure you ❤️
i love you ❤️
funny lol 😄
lets play with a ball ⚾
food is ready 🍴
today is not good 😄


Notice how the model doesn't account for word order, which is completely normal.

In [33]:
print('           '+ label_to_emoji(0)+ '    ' + label_to_emoji(1) + '    ' +  label_to_emoji(2)+ '    ' + label_to_emoji(3)+'   ' + label_to_emoji(4))
print(pd.crosstab(Y_test, pred_test.reshape(56,), rownames=['Actual'], colnames=['Predicted'], margins=True))

           ❤️    ⚾    😄    😞   🍴
Predicted  0.0  1.0  2.0  3.0  4.0  All
Actual                                 
0            6    0    0    1    0    7
1            0    8    0    0    0    8
2            1    0   17    0    0   18
3            1    0    2   13    0   16
4            0    0    0    0    7    7
All          8    8   19   14    7   56


We see that emojis related to emotions have a harder time to be mislabeled than other emojis.

## Emojifier-V2

This new model is supposed to accout for more complexe relationships, such as word orders.  
On top of the a word embedding layer, we'll use two layers of LSTM.

In [18]:
import numpy as np
import tensorflow
np.random.seed(0)
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input, Dropout, LSTM, Activation
from tensorflow.keras.layers import Embedding
np.random.seed(1)

This new model takes as input word indices and not the sentence as it is, we need a function to convert sentence to word indices.

In [20]:
def sentences_to_indices(X, word_to_index, max_len):
    
    m = X.shape[0]
    
    X_indices = np.zeros((m, max_len))
    
    for i in range(m):
        
        sentence_words = X[i].lower().split()
        
        j = 0
        
        for w in sentence_words:
            if w in word_to_index:
                X_indices[i, j] = word_to_index[w]
                j += 1
            
    return X_indices

Let's see if our function words as intended.

In [21]:
X1 = np.array(["funny lol", "lets play baseball", "food is ready for you"])
X1_indices = sentences_to_indices(X1, word_to_index, max_len=5)
print("X1 =", X1)
print("X1_indices =\n", X1_indices)

X1 = ['funny lol' 'lets play baseball' 'food is ready for you']
X1_indices =
 [[155345. 225122.      0.      0.      0.]
 [220930. 286375.  69714.      0.      0.]
 [151204. 192973. 302254. 151349. 394475.]]


Next, we need to create our custom Keras embedding layer that uses the GloVe words representation.

In [22]:
def pretrained_embedding_layer(word_to_vec_map, word_to_index):
    
    vocab_size = len(word_to_index) + 1
    any_word = list(word_to_vec_map.keys())[0]
    emb_dim = word_to_vec_map[any_word].shape[0]
      
    emb_matrix = np.zeros((vocab_size, emb_dim))
    
    for word, idx in word_to_index.items():
        emb_matrix[idx, :] = word_to_vec_map[word]

    embedding_layer = Embedding(vocab_size, emb_dim)

    embedding_layer.build((None,))
    
    embedding_layer.set_weights([emb_matrix])
    
    return embedding_layer

In [23]:
embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
print("weights[0][1][1] =", embedding_layer.get_weights()[0][1][1])
print("Input_dim", embedding_layer.input_dim)
print("Output_dim",embedding_layer.output_dim)

weights[0][1][1] = 0.39031
Input_dim 400001
Output_dim 50


We have all the pieces required to build the model.

In [24]:
def Emojify_V2(input_shape, word_to_vec_map, word_to_index):

    sentence_indices = Input(shape=input_shape, dtype='int32')
    
    embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
    
    embeddings = embedding_layer(sentence_indices)
    
    X = LSTM(128, return_sequences=True)(embeddings)
    X = Dropout(0.5)(X)
    X = LSTM(128)(X)
    X = Dropout(0.5)(X)
    X = Dense(5)(X)
    X = Activation('softmax')(X)
    
    model = Model(sentence_indices, X)
    
    return model

In [25]:
model = Emojify_V2((maxLen,), word_to_vec_map, word_to_index)
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 10)]              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 10, 50)            20000050  
_________________________________________________________________
lstm (LSTM)                  (None, 10, 128)           91648     
_________________________________________________________________
dropout (Dropout)            (None, 10, 128)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 5)                

In [26]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Before fitting the model, we need to preprocess our data.

In [27]:
X_train_indices = sentences_to_indices(X_train, word_to_index, maxLen)
Y_train_oh = convert_to_one_hot(Y_train, C = 5)

In [28]:
model.fit(X_train_indices, Y_train_oh, epochs = 50, batch_size = 32, shuffle=True)

Epoch 1/50
5/5 [==============================] - 2s 373ms/step - loss: 1.5936 - accuracy: 0.2348
Epoch 2/50
5/5 [==============================] - 2s 361ms/step - loss: 1.4850 - accuracy: 0.3333
Epoch 3/50
5/5 [==============================] - 2s 346ms/step - loss: 1.4442 - accuracy: 0.4015
Epoch 4/50
5/5 [==============================] - 2s 346ms/step - loss: 1.3602 - accuracy: 0.4773
Epoch 5/50
5/5 [==============================] - 2s 343ms/step - loss: 1.2044 - accuracy: 0.5985
Epoch 6/50
5/5 [==============================] - 2s 367ms/step - loss: 1.0520 - accuracy: 0.6818
Epoch 7/50
5/5 [==============================] - 2s 345ms/step - loss: 0.8450 - accuracy: 0.7273
Epoch 8/50
5/5 [==============================] - 2s 339ms/step - loss: 0.6870 - accuracy: 0.7576
Epoch 9/50
5/5 [==============================] - 2s 343ms/step - loss: 0.5656 - accuracy: 0.7803
Epoch 10/50
5/5 [==============================] - 2s 345ms/step - loss: 0.4747 - accuracy: 0.8561
Epoch 11/50
5/5 [==

In [29]:
X_test_indices = sentences_to_indices(X_test, word_to_index, max_len = maxLen)
Y_test_oh = convert_to_one_hot(Y_test, C = 5)
loss, acc = model.evaluate(X_test_indices, Y_test_oh)
print()
print("Test accuracy = ", acc)

2/2 [==============================] - 0s 2ms/step - loss: 1.1109 - accuracy: 0.8036

Test accuracy =  0.8035714030265808


Let's see where our model got it wrong.

In [31]:
C = 5
y_test_oh = np.eye(C)[Y_test.reshape(-1)]
X_test_indices = sentences_to_indices(X_test, word_to_index, maxLen)
pred = model.predict(X_test_indices)
for i in range(len(X_test)):
    x = X_test_indices
    num = np.argmax(pred[i])
    if(num != Y_test[i]):
        print('Expected emoji:'+ label_to_emoji(Y_test[i]) + ' prediction: '+ X_test[i] + label_to_emoji(num).strip())

Expected emoji:😄 prediction: he got a very nice raise	❤️
Expected emoji:😄 prediction: she got me a nice present	❤️
Expected emoji:😞 prediction: work is hard	😄
Expected emoji:😞 prediction: This girl is messing with me	❤️
Expected emoji:😞 prediction: work is horrible	😄
Expected emoji:🍴 prediction: any suggestions for dinner	😄
Expected emoji:😄 prediction: you brighten my day	❤️
Expected emoji:😞 prediction: she is a bully	❤️
Expected emoji:😞 prediction: My life is so boring	❤️
Expected emoji:😄 prediction: will you be my valentine	❤️
Expected emoji:😞 prediction: go away	⚾


Perhaps, the reason we got these misclassifications is due to the fact that our pretrained word embeddings weren't large enough to cover the meaning of all the words we have in our sentences.  
Now, we can try the model on a custom input sentence.

In [32]:
x_test = np.array(['What are we eating for dinner ?'])
X_test_indices = sentences_to_indices(x_test, word_to_index, maxLen)
print(x_test[0] +' '+  label_to_emoji(np.argmax(model.predict(X_test_indices))))

What are we eating for dinner ? 🍴


This new version of the model does account for word order and performs better than the baseline.